In [2]:
import json, os
import shutil
import pandas as pd

In [3]:
def preprocess_tcga_data(json_file, source_dirctory, target_dirctory):

    if not json_file or json_file == '':
        return False
    if not os.path.exists(source_dirctory):
        return False
    if not os.path.exists(target_dirctory):
        os.makedirs(target_dirctory)

    # read json_file
    with open(json_file, 'r') as jf:
        json_data = jf.read()

    json_info = json.loads(json_data)

    for file_info in json_info:
        file_id = file_info['file_id']
        file_name = file_info['file_name']
        submit_id = file_info['associated_entities'][0]['entity_submitter_id']
        source_file = source_dirctory + '/' + file_id + '/' + file_name
        target_file = target_dirctory + '/' + submit_id + '.tsv'
        shutil.copy(source_file, target_file)
        print('file_name :{} copy done'.format(file_name))

# preprocess_tcga_data("D:\BTp\data\metadata.cart.2023-03-02.json","D:\BTp\data","D:\BTp\processdata")

In [4]:
import os
#remove unecessary data 
def prodata(source_dirctory, target_dirctory):
    if not os.path.exists(source_dirctory):
        return False
    if not os.path.exists(target_dirctory):
        os.makedirs(target_dirctory)
    files = os.listdir(source_dirctory)

    for file in files:
        if file.endswith('.tsv'):
            file_path = os.path.join(source_dirctory, file)
            df = pd.read_csv(file_path, sep='\t', header=None, skiprows=[
                             0, 2, 3, 4, 5], usecols=[0, 1, 2, 3])
            f = os.path.splitext(file)[0]
            df.to_csv(target_dirctory+'/'+f, index=False, header=False)
            # print(f," done")

prodata("D:\BTp\processdata","D:\BTp\Process1data")




In [3]:
#collect all genes information from gtf file
GFF3 = pd.read_csv(
    filepath_or_buffer='Homo_sapiens.GRCh38.103.gtf',
    sep='\t',
    header=None,
    names=['seqid', 'source', 'type', 'start', 'end',
           'score', 'strand', 'phase', 'attributes'],
    skiprows=[0, 1, 2, 3, 4]
)
GFF3 = GFF3[GFF3['source'].notnull()]

GFF3 = GFF3[GFF3['type'] == 'gene']

GFF3['gene_id'] = GFF3.apply(lambda x: x.attributes.split(
    ';')[0].strip().split(' ')[1].strip('"'), axis=1)
GFF3['gene_name'] = GFF3.apply(lambda x: x.attributes.split(
    ';')[2].strip().split(' ')[1].strip('"'), axis=1)
GFF3['gene_biotype'] = GFF3.apply(lambda x: x.attributes.split(
    ';')[4].strip().split(' ')[1].strip('"'), axis=1)

GFF3 = pd.DataFrame(GFF3, columns=['gene_id', 'gene_name', 'gene_biotype'])
GFF3

C:\Users\Sachin Meena\AppData\Local\Temp\ipykernel_11720\840328527.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  GFF3 = pd.read_csv(


,gene_id,gene_name,gene_biotype
0,ENSG00000223972,DDX11L1,transcribed_unprocessed_pseudogene
12,ENSG00000227232,WASH7P,unprocessed_pseudogene
25,ENSG00000278267,MIR6859-1,miRNA
28,ENSG00000243485,MIR1302-2HG,lncRNA
36,ENSG00000284332,MIR1302-2,miRNA
...,...,...,...
3074189,ENSG00000271254,AC240274.1,protein_coding
3074340,ENSG00000275405,U1,snRNA
3074343,ENSG00000275987,U1,snRNA
3074346,ENSG00000277475,AC213203.2,protein_coding


In [6]:
cv=0
for file_name in os.listdir('D:\BTp\Process1data'):
    if file_name.endswith('.csv'):
        cv+=1
    else :
        file_path = os.path.join('D:\BTp\Process1data', file_name)
        df = pd.read_csv(file_path, header=None,skiprows=1)
        df.drop(df.columns[[1,2]], axis=1, inplace=True)
        df.to_csv(file_path, header=False, index=False)

In [6]:

def all_data(source_dirctory, df):
    if not os.path.exists(source_dirctory):
        return False
    files = os.listdir(source_dirctory)
    
    for file in files:
        
        data = pd.read_csv(filepath_or_buffer=source_dirctory + '/' + file,sep=',', names=['gene_id', file])
        data['gene_id'] = data.apply(lambda x: x.gene_id.split('.')[0], axis=1)
        df = pd.merge(df,data, on='gene_id')
        print(file,' merge completed!')
    return df


dff = all_data("D:\BTp\Process1data", GFF3)

TCGA-05-4244-01A-01R-1107-07  merge completed!
TCGA-05-4249-01A-01R-1107-07  merge completed!
TCGA-05-4250-01A-01R-1107-07  merge completed!
TCGA-05-4382-01A-01R-1206-07  merge completed!
TCGA-05-4384-01A-01R-1755-07  merge completed!
TCGA-05-4389-01A-01R-1206-07  merge completed!
TCGA-05-4390-01A-02R-1755-07  merge completed!
TCGA-05-4395-01A-01R-1206-07  merge completed!
TCGA-05-4396-01A-21R-1858-07  merge completed!
TCGA-05-4397-01A-01R-1206-07  merge completed!
TCGA-05-4398-01A-01R-1206-07  merge completed!
TCGA-05-4402-01A-01R-1206-07  merge completed!
TCGA-05-4403-01A-01R-1206-07  merge completed!
TCGA-05-4405-01A-21R-1858-07  merge completed!
TCGA-05-4410-01A-21R-1858-07  merge completed!
TCGA-05-4415-01A-22R-1858-07  merge completed!
TCGA-05-4417-01A-22R-1858-07  merge completed!
TCGA-05-4418-01A-01R-1206-07  merge completed!


MemoryError: Unable to allocate 353. MiB for an array with shape (2, 23129217) and data type object

In [ ]:
df.to_csv('mergedgenes1.csv')

In [ ]:
#filter only protein coding genes
df = pd.read_csv('mergedgenes1.csv')
df = df[df['gene_biotype'] == 'protein_coding']